# 1. Carga y exploración de datos (2 puntos)
# • Carga el dataset proporcionado en Spark.
# • Convierte los datos en un RDD y un DataFrame.
# • Explora los datos: muestra las primeras filas, el esquema y genera estadísticas descriptivas.



In [22]:
# Importar la clase SparkSession de pyspark.sql
from pyspark.sql import SparkSession

# Crear una instancia de SparkSession
spark = SparkSession.builder \
    .appName("AnalisisMigratorio") \
    .getOrCreate()

# Cargar el archivo CSV en un DataFrame. 'header=True' para usar la primera fila como encabezado y 'inferSchema=True' para que Spark intente adivinar
# los tipos de datos.
df_migraciones = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("migraciones.csv")

# Mostrar los datos del DataFrame
print("Datos del DataFrame: Migraciones.csv")
df_migraciones.show()
df_migraciones = df_migraciones.withColumnRenamed("Razón", "Razon")

# Convertir el DataFrame a un RDD
rdd_migraciones = df_migraciones.rdd

# Mostrar los datos del RDD
print("Datos del RDD: Migraciones.rdd")
rdd_migraciones.take(5)

# Mostrar las primeras 5 filas del DataFrame
print("Primeras filas del DataFrame:")
df_migraciones.show(5)

# Mostrar el esquema del DataFrame para ver los tipos de datos de cada columna
print("Esquema del DataFrame:")
df_migraciones.printSchema()

# Generar y mostrar estadísticas descriptivas para las columnas numéricas
print("Estadísticas descriptivas del DataFrame:")
df_migraciones.describe().show()



Datos del DataFrame: Migraciones.csv
+---+---------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
| ID|   Origen|        Destino| Año|    Razón|PIB_Origen|PIB_Destino|Tasa_Desempleo_Origen|Tasa_Desempleo_Destino|Nivel_Educativo_Origen|Nivel_Educativo_Destino|Población_Origen|Población_Destino|
+---+---------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
|  1|   México|           EEUU|2015|Económica|      8900|      62000|                  5.2|                   3.8|                   8.5|                   12.3|       125000000|        331000000|
|  2|    Siria|       Alemania|2016|Conflicto|      2500|      45000|                 15.4|                   4.5|                   6.2|                   14.1|        180000

# 2. Procesamiento de datos con RDDs y DataFrames (3 puntos)
# • Aplica transformaciones sobre los RDDs (filter, map, flatMap).
# • Aplica acciones sobre los RDDs (collect, take, count).
# • Realiza operaciones con DataFrames: filtrado, agregaciones y ordenamiento.
# • Escribe los resultados en formato Parquet.

In [26]:
# Usar la transformación 'filter' para obtener las migraciones por conflictos
# La función 'lambda' filtra las filas donde la columna 'Razón' es "Conflicto"
rdd_conflictos = rdd_migraciones.filter(lambda row: row['Razon'] == 'Conflicto')

# Usar la acción 'count' para saber cuántas migraciones son por conflicto
print(f"Número de migraciones por conflicto: {rdd_conflictos.count()}")

# Usar la acción 'take' para ver las primeras 2 migraciones por conflicto
print("Primeras 2 migraciones por conflicto:")
print(rdd_conflictos.take(2))

# Se usa 'map' para crear un RDD con tuplas (Origen, Destino) para cada migración.
rdd_origen_destino = rdd_migraciones.map(lambda row: (row['Origen'], row['Destino']))
print("\nPrimeras 3 tuplas (Origen, Destino) del RDD:")
print(rdd_origen_destino.take(3))

# Se usa 'flatMap' para crear un RDD de palabras a partir de las razones de migración. Esto es útil para análisis de texto.
rdd_razones_flat = rdd_migraciones.flatMap(lambda row: row['Razon'].split())
print("\nPrimeras 5 palabras del RDD 'flatMap':")
print(rdd_razones_flat.take(5))

# Filtrar las migraciones con un PIB de destino mayor a $40,000 sobre un DataFrame
df_high_pib = df_migraciones.filter(df_migraciones.PIB_Destino > 40000)
print("Migraciones con PIB de destino alto:")
df_high_pib.show()

# Agrupar y agregar datos: calcular el promedio del PIB de origen por razón de migración sobre un DataFrame
df_avg_pib_by_reason = df_migraciones.groupBy("Razon").agg({"PIB_Origen": "avg"})
print("Promedio del PIB de origen por razón de migración:")
df_avg_pib_by_reason.show()

# Ordenar los datos por población de origen de forma descendente sobre un DataFrame
df_sorted_by_population = df_migraciones.orderBy("Población_Origen", ascending=False)
print("Migraciones ordenadas por población de origen (descendente):")
df_sorted_by_population.show()

# Escribir el DataFrame 'df_avg_pib_by_reason' en formato Parquet
# El modo 'overwrite' sobrescribe el archivo si ya existe
df_avg_pib_by_reason.write.mode("overwrite").parquet("resultados_migracion.parquet")

Número de migraciones por conflicto: 1
Primeras 2 migraciones por conflicto:
[Row(ID=2, Origen='Siria', Destino='Alemania', Año=2016, Razon='Conflicto', PIB_Origen=2500, PIB_Destino=45000, Tasa_Desempleo_Origen=15.4, Tasa_Desempleo_Destino=4.5, Nivel_Educativo_Origen=6.2, Nivel_Educativo_Destino=14.1, Población_Origen=18000000, Población_Destino=83000000)]

Primeras 3 tuplas (Origen, Destino) del RDD:
[('México', 'EEUU'), ('Siria', 'Alemania'), ('Venezuela', 'Colombia')]

Primeras 5 palabras del RDD 'flatMap':
['Económica', 'Conflicto', 'Política', 'Laboral', 'Económica']
Migraciones con PIB de destino alto:
+---+------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
| ID|Origen|        Destino| Año|    Razon|PIB_Origen|PIB_Destino|Tasa_Desempleo_Origen|Tasa_Desempleo_Destino|Nivel_Educativo_Origen|Nivel_Educativo_Destino|Población_Origen|Población_Desti

# 3. Consultas con Spark SQL (2 puntos)
# • Registra el DataFrame como una tabla temporal.
# • Realiza consultas sobre los principales países de origen y destino.
# • Analiza las principales razones de migración por región

In [27]:
# Registrar el DataFrame como una tabla temporal llamada 'migraciones'
df_migraciones.createOrReplaceTempView("migraciones")

# Consultar los principales países de origen
print("Principales países de origen:")
spark.sql("SELECT Origen, COUNT(*) as Frecuencia FROM migraciones GROUP BY Origen ORDER BY Frecuencia DESC").show()

# Consultar los principales países de destino
print("Principales países de destino:")
spark.sql("SELECT Destino, COUNT(*) as Frecuencia FROM migraciones GROUP BY Destino ORDER BY Frecuencia DESC").show()

# Consultar y contar las diferentes razones de migración
print("Conteo de razones de migración:")
spark.sql("SELECT Razon, COUNT(*) as Frecuencia FROM migraciones GROUP BY Razon ORDER BY Frecuencia DESC").show()

Principales países de origen:
+---------+----------+
|   Origen|Frecuencia|
+---------+----------+
|Argentina|         1|
|    Siria|         1|
|    India|         1|
|Venezuela|         1|
|   México|         1|
+---------+----------+

Principales países de destino:
+---------------+----------+
|        Destino|Frecuencia|
+---------------+----------+
|         España|         1|
|Emiratos Árabes|         1|
|       Alemania|         1|
|           EEUU|         1|
|       Colombia|         1|
+---------------+----------+

Conteo de razones de migración:
+---------+----------+
|    Razon|Frecuencia|
+---------+----------+
|Económica|         2|
| Política|         1|
|Conflicto|         1|
|  Laboral|         1|
+---------+----------+



# 4. Aplicación de MLlib para predicción de flujos migratorios (3 puntos)
# • Convierte los datos en un formato adecuado para MLlib.
# • Aplica un modelo de regresión logística para predecir la probabilidad de migración basada en factores socioeconómicos.
# • Evalúa el modelo y analiza su precisión.

In [29]:
# Importar las clases necesarias de MLlib
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

# Para la predicción, se necesita un objetivo ('label') y características ('features').
# Aquí, la 'Razón' podría ser un buen objetivo, categorizado como binario (e.g., "Económica" vs. no "Económica").

# Codificar la variable categórica 'Razón' a un índice numérico
indexer = StringIndexer(inputCol="Razon", outputCol="RazonIndex")

# Crear una variable dummy a partir del índice
encoder = OneHotEncoder(inputCol="RazonIndex", outputCol="RazonVector")

# Se seleccionarán las características numéricas para el modelo
features = ['PIB_Origen', 'Tasa_Desempleo_Origen', 'Nivel_Educativo_Origen', 'Población_Origen']

# Combinar las características en un solo vector
vector_assembler = VectorAssembler(inputCols=features, outputCol="features")

# Se podría crear una columna binaria, por ejemplo 'EsEconómica', para la predicción
# Esta columna será nuestro 'label'
df_ml = df_migraciones.withColumn("EsEconómica", (df_migraciones["Razon"] == "Económica").cast("int"))

# Ensamblar las características y dividir los datos para entrenamiento y prueba
pipeline = Pipeline(stages=[vector_assembler])
df_ml = pipeline.fit(df_ml).transform(df_ml)
train_data, test_data = df_ml.randomSplit([0.7, 0.3], seed=42)

# Crear una instancia del modelo de Regresión Logística
# Se usa la columna 'EsEconómica' como la etiqueta (label)
lr = LogisticRegression(featuresCol="features", labelCol="EsEconómica", predictionCol="prediccion")

# Entrenar el modelo con los datos de entrenamiento
lr_model = lr.fit(train_data)

# Realizar predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)

# Importar la clase para la evaluación de clasificadores binarios
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluar el modelo usando el área bajo la curva de la ROC (AUC)
evaluator = BinaryClassificationEvaluator(labelCol="EsEconómica", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Área Bajo la Curva (AUC): {auc}")

# Para obtener la precisión, se puede usar otra métrica
# Calcular la precisión manualmente
predictions.select("EsEconómica", "prediccion").show()
correct_predictions = predictions.filter(predictions.EsEconómica == predictions.prediccion).count()
total_predictions = predictions.count()
accuracy = correct_predictions / total_predictions
print(f"Precisión del modelo: {accuracy}")

# Finalizar la sesión de Spark
spark.stop()

Área Bajo la Curva (AUC): 0.0
+-----------+----------+
|EsEconómica|prediccion|
+-----------+----------+
|          0|       0.0|
+-----------+----------+

Precisión del modelo: 1.0
